<a href="https://colab.research.google.com/github/Lifeisbetterwithlena/SCM_Fallstudie/blob/main/Kopie_von_1_3_Fallstudie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie Flink
## Standortentscheidung Erweiterung 

## Modell

### Indexmengen
$s \in S$ : Menge der potenziellen DarkStores (Standorte)

$i \in I$ : Menge der (Nachfrage) i-Koordinaten 

$j \in J$ : Menge der (Nachfrage) j-Koordinaten  



### Parameter
$n_{ij}$ : Nachfrage an $i$$j$

$u_{s}$ : Lagerumschlagsleistung an Standort $s$

$c_{s}$ : Errichtungskosten für Standort $s$

$ki_{s}$ : $i$-Kooridnate von Standort $s$ 

$kj_{s}$ : $j$-Kooridnate von Standort $s$ 



### Entscheidungsvariablen

$V_{sij} \in \{0,1\}$ : Binäre Standort-Versorgungsvariable

$Y_{s} \in \{0,1\}$ : Binäre Standortausbauvariable

### ----------
#### Erweiterung 

$NA_{sij} \ge 0$ : Nachfrageabdeckung von Standort $S$ an Quadrant  $ij$ 

$Q_{ij} \in \{0,1\}$ : Binäre Versorgungsvariable (Quadrant)


### Neue Zielfunktion

Maximiere Nachfrageabdeckung erweiterung:

Max $NA = \sum_{s,i,j} (NA_{s,i,j})$  

### Nebenbedingungen

**(1) Budget einhalten**

$\sum_{s} (c_s*Y_s) \le 1.000.000 $

Summe über: 

Kosten  für Ausbau * Entscheidung Ausbau <= Budget

**(2) Lieferzeit einhalten**

$(|ki_s - i| + |kj_s - j|)* V_{s,i,j} \le 5 $

$∀ s,i,j$


Prüfe für jedes $s, i , j$:

Entfernung  <= 5, wenn $ij$ von $s$ beliefert wird

**(3) Höchstens Doppelbedienung der Quandranden**

$\sum_s V_{s,i,j} \le 2$

$∀ i, j$

Prüfe für alle Koordinaten, ob Belieferung kleiner gleich 1.

**(4) Kapazitäten einhalten**

$\sum_{i,j} (NA_{s,i,j}) \le u_s *Y_s $ 

$∀ s$

Prüfe für jeden Standort:

Summe Versorgung (ja/nein) * Nachfrageabdeckung  ist kleiner (gleich) als Umschlagsleistung. 

**(5) Nachfragebedienung**

$\sum_s NA_{s,i,j} = n_{i,j} *Q_{i,j}$ 

$∀ i,j$

Summe aus der Nachfrageabdeckung aller Standorte für jede i und j ergibt die Nachfrge in i j, wenn der Quadrant bedient wird

**(6) Nachfrageabdeckung nur über 0 wenn Versorgungsvariable 1**

$NA_{s,i,j} <= n_{i,j} * V_{s,i,j}$ 

$∀ s,i,j$



**(7) Nachfrageabdeckung von Standort kleiner gleich Quadantversorgung**

$Y_{s,i,j} <= Q_{i,j} $ 

$∀ s,i,j$



## Implementierung

## Datenaufbereitung


1.   Fallstudien-Daten in Google-Drive laden
2.   Google-Drive mit Colab-Notebook verbinden
3.   Daten mit `pandas` laden



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Ordner finden
! ls drive/MyDrive/Industrielles_Management/Daten/Fallstudie

Nachfrage.csv  Standorte.csv


In [3]:
# Pfad zurückgeben
! cd drive/MyDrive/Industrielles_Management/Daten/Fallstudie && pwd

/content/drive/MyDrive/Industrielles_Management/Daten/Fallstudie


In [4]:
# Daten laden
import pandas as pd

In [5]:
path = "/content/drive/MyDrive/Industrielles_Management/Daten/Fallstudie"

In [6]:
# Nachfragedaten lesen & speichern
nachfrage_df = pd.read_csv(f"{path}/Nachfrage.csv", sep=";")

In [7]:
# Nachfragedaten ausgeben
nachfrage_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,7,91,34,77,33,47,43,51,81,28,88,74,81
1,1,73,82,76,51,80,42,83,19,37,37,85,18,67
2,2,85,57,67,14,1,92,41,32,16,83,65,39,18
3,3,42,87,23,56,15,99,60,85,90,68,9,63,88
4,4,23,93,36,65,96,31,8,54,27,74,59,70,36
5,5,2,71,34,38,62,45,86,95,93,12,83,59,17
6,6,31,41,54,1,97,73,5,73,44,93,94,2,56
7,7,61,44,86,1,69,65,32,95,61,39,7,61,31
8,8,59,49,1,70,86,26,62,40,96,82,3,20,83
9,9,10,38,37,77,16,94,34,67,96,64,32,30,63


In [8]:

# nachfrage is a dictionary of nachfrage_df. The first key are the values of column 1 (i coordinate) 
# the second key are the values of the columns (j coordinate)
# skip the first column for the second key since it's not an j cooridnate

nachfrage = {int(row[0]): {int(col): value for col, value in row.items() if col != 'Unnamed: 0'} for _, row in nachfrage_df.iterrows()}

nachfrage = {int(row[0]): {int(col): value for col, value in row.items() if col != 'Unnamed: 0'} for _, row in nachfrage_df.iterrows()}

# test if it worked

# nachfrage i=0 & j=7
print(nachfrage[0][7])

#nachfrage i=10 & j=12
print(nachfrage[10][12])

print(nachfrage[12][12])


51
43
83


In [9]:
# Standortdaten lesen & speichern
standorte = pd.read_csv(f"{path}/Standorte.csv", sep=";", decimal=",")

In [10]:
# Standortdaten ausgeben
standorte

,Potenzielle_Standorte,i_Koordinate,j_Koordinate,Lagerumschlagleistung,Errichtungskosten
0,0,1,0,500,100000
1,1,8,11,600,90000
2,2,10,4,500,95000
3,3,10,10,750,101000
4,4,13,6,725,98500
5,5,4,8,750,97500
6,6,13,7,455,105000
7,7,4,6,3000,225000
8,8,12,6,1400,300000
9,9,4,4,2250,200000


## Indexmengen

In [11]:
# Standorte
S = standorte["Potenzielle_Standorte"].unique().tolist() # Menge der Standorte


In [12]:
# Ausgabe Standorte
S

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [13]:
# I Kooritnaten
I = [key for key, value in nachfrage.items()]

print('I-Koordinaten: ' + str(I))

I-Koordinaten: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [14]:
# J Koordinaten
J = [list(value.keys()) for value in nachfrage.values()][0]

print('J-Koordinaten: ' + str(J))


J-Koordinaten: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


## Parameter

In [15]:
# Standortkoordinaten

# I-Koordinate von Standort S
ki = standorte.set_index(["Potenzielle_Standorte"]).to_dict("dict")["i_Koordinate"]

# J-Koordinate von Standort S
kj = standorte.set_index(["Potenzielle_Standorte"]).to_dict("dict")["j_Koordinate"]

# check
print(ki)

print(kj)

{0: 1, 1: 8, 2: 10, 3: 10, 4: 13, 5: 4, 6: 13, 7: 4, 8: 12, 9: 4}
{0: 0, 1: 11, 2: 4, 3: 10, 4: 6, 5: 8, 6: 7, 7: 6, 8: 6, 9: 4}


In [16]:
# Umschlagsleistung
u = standorte.set_index(["Potenzielle_Standorte"]).to_dict("dict")["Lagerumschlagleistung"]
#check
u


{0: 500,
 1: 600,
 2: 500,
 3: 750,
 4: 725,
 5: 750,
 6: 455,
 7: 3000,
 8: 1400,
 9: 2250}

In [17]:
# Errichtungskosten
c = standorte.set_index(["Potenzielle_Standorte"]).to_dict("dict")["Errichtungskosten"]
#check
c

{0: 100000,
 1: 90000,
 2: 95000,
 3: 101000,
 4: 98500,
 5: 97500,
 6: 105000,
 7: 225000,
 8: 300000,
 9: 200000}

In [18]:
# Nachfrage
n = nachfrage
#check
n[0][0]

7

## Solver

In [19]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 24.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.12 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.


In [20]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')
# Erstelle einen Solver
#solver = pywraplp.Solver('solver_fallstudie', pywraplp.Solver.GLPK_MIXED_INTEGER_PROGRAMMING)


## Entscheidungsvariablen

In [21]:
# Entscheidung über Versorgung von Quandrand ij

Q={}
for i in I:
  for j in J:
        Q[i,j] = solver.BoolVar(f"{i},{j}")

#check
Q[1,0]


1,0

In [22]:
# Binäre Versorgungsvariable
V={}
for s in S: 
  for i in I:
    for j in J:
        V[s,i,j] = solver.BoolVar(f"{s},{i},{j}")

#check
V[1,3,7]

1,3,7

In [23]:
# Binäre Standortausbauvariable
Y={}
for s in S:
  Y[s] = solver.BoolVar(f"{s}")


#check
Y[1]

1

In [24]:
# Nachfrageabdeckung 

#Obergrenze unendlich (float)
infinity = solver.infinity()

NA={}
for s in S:
  for i in I:
      for j in J:
        NA[s,i,j] = solver.IntVar(0, infinity, f"{s},{i},{j}")


In [25]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 3559


## Zielfunktion

### Neue Zielfunktion

Maximiere Nachfrageabdeckung erweiterung:

Max $NA = \sum_{s,i,j} (NA_{s,i,j})$  


In [26]:
solver.Maximize(sum(NA[s,i,j] for s in S for i in I for j in J))

## Nebenbedingungen

**(1) Budget einhalten**

$\sum_{s} (c_s*Y_s) \le 1.000.000 $

Summe über: 

Kosten  für Ausbau * Entscheidung Ausbau <= Budget

In [27]:

solver.Add(sum(c[s]*Y[s] for s in S)<= 1000000)
 

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f2b9f74e0f0> >

**(2) Lieferzeit einhalten**

$(|ki_s - i| + |kj_s - j|)* v_{s,i,j} \le 5 $

$∀ s,i,j$


Prüfe für jedes $s, i , j$:

Entfernung  <= 5, wenn $ij$ von $s$ beliefert wird

In [28]:
for s in S:
 for i in I:
    for j in J:
      solver.Add(((abs(ki[s]-i) +abs(kj[s]- j))*V[s,i,j])<=5)


**(3) Höchstens Doppelbedienung der Quandranden**

$\sum_s V_{s,i,j} \le 2$

$∀ i, j$

Prüfe für alle Koordinaten, ob Belieferung kleiner gleich 1.

In [29]:
for i in I:
  for j in J:
    solver.Add(sum(V[s,i,j] for s in S)<=2)

**(4) Kapazitäten einhalten**

$\sum_{i,j} (NA_{s,i,j}) \le u_s *Y_s $

$∀ s$

Prüfe für jeden Standort:

Summe Versorgung (ja/nein) * Nachfrage  ist kleiner (gleich) als Umschlagsleistung. 

In [30]:
for s in S:
  solver.Add(sum(NA[s,i,j] for i in I for j in J)<= (u[s]* Y[s]))



**(5) Nachfragebedienung**

$\sum_s NA_{s,i,j} = n_{i,j} *Q_{i,j}$ 

$∀ i,j$

Summe aus der Nachfrageabdeckung aller Standorte für jede i und j ergibt die Nachfrge in i j, wenn der Quadrant bedient wird

In [31]:
for i in I:
  for j in J:
    solver.Add(sum(NA[s,i,j] for s in S) == (n[i][j]*Q[i,j]))

**(6) Nachfrageabdeckung nur über 0 wenn Versorgungsvariable 1**

$NA_{s,i,j} <= n_{i,j} * V_{s,i,j}$ 

$∀ s,i,j$



In [32]:
for s in S:
  for i in I:
    for j in J:
      solver.Add(NA[s,i,j] <= (n[i][j]* V[s,i,j]))

**(7) Nachfrageabdeckung von Standort kleiner gleich Quadantversorgung**

$Y_{s,i,j} <= Q_{i,j} $ 

$∀ s,i,j$



In [33]:
for s in S:
  for i in I:
    for j in J:
      solver.Add(V[s,i,j] <= Q[i,j])

In [34]:
print("Anzahl an Nebenbedingungen: ", solver.NumConstraints())

Anzahl an Nebenbedingungen:  5419


## Berechnung Lösung

In [35]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
  print('LÖSUNG:')
  print('Zielfunktionswert (Gesamte Nachfrageabdeckung) =', solver.Objective().Value())
else:
  print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Gesamte Nachfrageabdeckung) = 7481.999999999999


### Standortausbau

In [36]:
# Gebe die Gesamtkosten für den Ausbau an

cost= 0
for s in S:
  if(Y[s].solution_value()==1):
    print("Wir bauen den Standort " + str(s) + " aus.")
    cost += c[s]

print("")
print("Die Gesamtkosten für den Ausbau betragen " + str(cost) + " €")
  

Wir bauen den Standort 1 aus.
Wir bauen den Standort 2 aus.
Wir bauen den Standort 3 aus.
Wir bauen den Standort 4 aus.
Wir bauen den Standort 5 aus.
Wir bauen den Standort 7 aus.
Wir bauen den Standort 9 aus.

Die Gesamtkosten für den Ausbau betragen 907000 €


### Belieferungsplan

In [37]:
na= 0

for s in S:
  if( Y[s].solution_value()==1):
    print("")
    print("")
    print("Belieferung durch Standort " + str(s) +":")
    print("")
    na_s = 0
    for i in I:
      for j in J:
        if(V[s,i,j].solution_value()==1):
          abstand= (abs(ki[s]-i) +abs(kj[s]- j))
          if(NA[s,i,j].solution_value()>0):
            print("Quadrand: " +str(i) + "/" +str(j) + " Abstand: " + str(abstand) + " NA in Prozent: " + str(round((NA[s,i,j].solution_value()*100)/n[i][j],2)) + "% " + "NA absolut: " +  str(NA[s,i,j].solution_value()))
            na_s += NA[s,i,j].solution_value()
    print("")
    print("Kapazität im Standort: " + str(u[s]))
    print("Nachfrageabdeckung durch Standort: " + str(na_s))
    na += na_s

print("")
print("Nachfrageabdeckung insgesamt: " +str(na))

          



Belieferung durch Standort 1:

Quadrand: 3/11 Abstand: 5 NA in Prozent: 100.0% NA absolut: 63.0
Quadrand: 4/12 Abstand: 5 NA in Prozent: 100.0% NA absolut: 36.0
Quadrand: 5/11 Abstand: 3 NA in Prozent: 1.69% NA absolut: 1.0
Quadrand: 6/12 Abstand: 3 NA in Prozent: 100.0% NA absolut: 56.0
Quadrand: 7/12 Abstand: 2 NA in Prozent: 100.0% NA absolut: 31.0
Quadrand: 8/8 Abstand: 3 NA in Prozent: 100.0% NA absolut: 96.0
Quadrand: 9/7 Abstand: 5 NA in Prozent: 100.0% NA absolut: 67.0
Quadrand: 9/8 Abstand: 4 NA in Prozent: 28.12% NA absolut: 27.0
Quadrand: 9/10 Abstand: 2 NA in Prozent: 100.0% NA absolut: 32.0
Quadrand: 10/11 Abstand: 2 NA in Prozent: 100.0% NA absolut: 73.0
Quadrand: 10/12 Abstand: 3 NA in Prozent: 18.6% NA absolut: 8.0
Quadrand: 11/10 Abstand: 4 NA in Prozent: 100.0% NA absolut: 28.0
Quadrand: 12/12 Abstand: 5 NA in Prozent: 98.8% NA absolut: 82.0

Kapazität im Standort: 600
Nachfrageabdeckung durch Standort: 600.0


Belieferung durch Standort 2:

Quadrand: 8/1 Abstand: 5

### Nachfrageabdeckung je Quadrand

In [38]:
# Wenn Quadrant versorgt wird, gebe aus, wie viel Prozent der Nachfrage im Quadrant bedient wird

for i in I:
  for j in J:
    if(Q[i,j].solution_value()==1):
      print("Wir versorgen den Quadranten "+ str(i) + "/" + str(j)+ " zu " + str((sum(NA[s,i,j].solution_value()for s in S)*100)/n[i][j]) +"%" )

Wir versorgen den Quadranten 0/3 zu 100.0%
Wir versorgen den Quadranten 0/4 zu 100.0%
Wir versorgen den Quadranten 0/5 zu 100.0%
Wir versorgen den Quadranten 0/6 zu 100.0%
Wir versorgen den Quadranten 0/7 zu 100.0%
Wir versorgen den Quadranten 0/8 zu 100.0%
Wir versorgen den Quadranten 0/9 zu 100.0%
Wir versorgen den Quadranten 1/2 zu 100.0%
Wir versorgen den Quadranten 1/3 zu 100.0%
Wir versorgen den Quadranten 1/4 zu 100.0%
Wir versorgen den Quadranten 1/5 zu 100.0%
Wir versorgen den Quadranten 1/6 zu 100.0%
Wir versorgen den Quadranten 1/7 zu 100.0%
Wir versorgen den Quadranten 1/8 zu 100.0%
Wir versorgen den Quadranten 1/9 zu 100.0%
Wir versorgen den Quadranten 1/10 zu 100.0%
Wir versorgen den Quadranten 2/1 zu 100.0%
Wir versorgen den Quadranten 2/2 zu 100.0%
Wir versorgen den Quadranten 2/3 zu 100.0%
Wir versorgen den Quadranten 2/4 zu 100.0%
Wir versorgen den Quadranten 2/5 zu 100.0%
Wir versorgen den Quadranten 2/6 zu 100.0%
Wir versorgen den Quadranten 2/7 zu 100.0%
Wir versor

### Standortauslastung

In [39]:
for s in S:
  na_s= 0
  kap = 0
  if(Y[s].solution_value()==1):
    na_s += sum(NA[s,i,j].solution_value() for i in I for j in J)
    kap= round((na_s*100)/u[s],0)
    print("Der Standort "+str(s) + " ist zu " + str(kap) + "% ausgelastet")


Der Standort 1 ist zu 100.0% ausgelastet
Der Standort 2 ist zu 100.0% ausgelastet
Der Standort 3 ist zu 100.0% ausgelastet
Der Standort 4 ist zu 100.0% ausgelastet
Der Standort 5 ist zu 100.0% ausgelastet
Der Standort 7 ist zu 100.0% ausgelastet
Der Standort 9 ist zu 51.0% ausgelastet


In [40]:
for s in S:
  for i in I:
    for j in J:
      if(NA[s,i,j].solution_value() != n[i][j] and NA[s,i,j].solution_value() != 0 ):
        print("Wir beliefern doppelt: " + str(i) + str(j))
   

Wir beliefern doppelt: 511
Wir beliefern doppelt: 98
Wir beliefern doppelt: 1012
Wir beliefern doppelt: 1212
Wir beliefern doppelt: 108
Wir beliefern doppelt: 106
Wir beliefern doppelt: 107
Wir beliefern doppelt: 1012
Wir beliefern doppelt: 1212
Wir beliefern doppelt: 106
Wir beliefern doppelt: 107
Wir beliefern doppelt: 108
Wir beliefern doppelt: 511
Wir beliefern doppelt: 98
Wir beliefern doppelt: 54
Wir beliefern doppelt: 54


In [41]:
for s in S:
  for i in I:
    for j in J:
      if(NA[s,i,j].solution_value()!=0):
        print(NA[s,i,j].solution_value())

63.0
36.0
1.0
56.0
31.0
96.0
67.0
27.0
32.0
73.0
8.0
28.0
82.0
49.0
10.0
38.0
37.0
77.0
56.0
35.0
21.0
89.0
9.0
25.0
35.0
9.0
10.0
39.0
61.0
20.0
83.0
64.0
30.0
63.0
68.0
12.0
68.0
35.0
41.0
4.0
65.0
24.0
72.0
1.0
94.0
28.0
60.0
22.0
9.0
56.0
32.0
25.0
86.0
17.0
42.0
61.0
37.0
52.0
66.0
10.0
13.0
15.0
81.0
28.0
37.0
85.0
65.0
39.0
88.0
58.0
17.0
94.0
7.0
82.0
69.0
47.0
43.0
51.0
80.0
42.0
83.0
19.0
37.0
14.0
1.0
92.0
41.0
32.0
16.0
83.0
56.0
15.0
99.0
60.0
85.0
90.0
68.0
9.0
93.0
65.0
96.0
31.0
8.0
27.0
74.0
59.0
70.0
34.0
61.0
45.0
86.0
95.0
93.0
12.0
83.0
1.0
97.0
73.0
5.0
73.0
44.0
93.0
69.0
32.0
95.0
61.0
26.0
62.0
40.0
34.0
77.0
33.0
76.0
51.0
57.0
67.0
42.0
87.0
23.0
23.0
36.0
54.0
2.0
71.0
38.0
1.0
41.0
54.0
86.0
1.0
65.0
70.0
86.0
16.0
